# Crawling Tweet for Topic

## Import Library

In [1]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
# import library: time
import time
now = time.localtime() # used to add timestamp to filename

## <Step1. API 호출> : 트위터 API로 데이터 가져오기

### [API 데이터로 데이터프레임 생성하기]

### 트위터 API 가져오기

In [3]:
#!pip install tweepy==3.10.0


In [4]:
# 패키지 설치 확인
import tweepy #ModuleNotFoundError: No module named 'tweepy'
tweepy.__version__

'3.10.0'

In [5]:
# 발급 완료된 키를 {your_key} 대신 입력합니다.
CONSUMER_KEY = "GyhYUuPz1n7G8xJinc6oS669Q"
CONSUMER_SECRET = "zYUmczSjM9r4yjUT6TKOFVNoAvjn84uZ8p2FtJGPYeTpAL83RB"
ACCESS_TOKEN_KEY = "1449194998998593543-f5yvwnxfwwAQj4YN35Ubgr1nJnUpk2"
ACCESS_TOKEN_SECRET = "iMbxkqQfnBkbrzKbPVbtT3peJpsWmxIS2UBnF3LbRKgSM"

In [6]:
# 개인정보 인증을 요청하는 Handler입니다.
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

In [7]:
# 인증 요청을 수행합니다.
auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)

In [8]:
# twitter API를 사용하기 위한 준비입니다.
api = tweepy.API(auth)

### 키워드 검색

In [9]:
# twitter API를 사용하여 ‘손흥민’이 포함된 트윗들을 크롤링한 뒤, entities 에서 ‘user_mentions’, ‘hashtags’를 추출합니다.
keyword = "라그나로크"
tweets = api.search(keyword) #Old version: replaced to search_tweets()
# tweets = api.search_tweets(keyword) #For ver 4.0
for tweet in tweets:
    print(tweet.text)
    print(tweet.entities['user_mentions'])
    print(tweet.entities['hashtags'])
    print(tweet.created_at)

RT @MCU_0range_: 토르 라그나로크에서 로키랑 발키리 전투씬이 기억에 남은 이유. 로키가 정신 지배 능력 사용할 수 있었다는 것 때문에 기억에 남았다. 뭐, 그러고 발키리한테 기절 당했지만 uu..
[{'screen_name': 'MCU_0range_', 'name': '오렌쥐🍊', 'id': 1443965247031705600, 'id_str': '1443965247031705600', 'indices': [3, 15]}]
[]
2021-10-24 04:36:09
토르 라그나로크에서 로키랑 발키리 전투씬이 기억에 남은 이유. 로키가 정신 지배 능력 사용할 수 있었다는 것 때문에 기억에 남았다. 뭐, 그러고 발키리한테 기절 당했지만 uu..
[]
[]
2021-10-24 04:30:34
왜.네가 이것을...라그나로크를 가지고 있는거지?/자동
[]
[]
2021-10-24 03:54:02
라그나로크라고?어떻게..../자동
[]
[]
2021-10-24 03:23:59
유독 라그나로크 비하인드가 많은 거 같은데... 가장 안 풀리긴 했지..... 저때 망토 찢어져서 반망토로 다니는게 너무 좋았던.... https://t.co/PmDVWNJDnx
[]
[]
2021-10-24 02:40:01
그럼, "라그나로크"를 일으키는 이 힘으로——
네가 비웃었던 파멸자의 힘으로 이 세상의 약자들을 지켜내겠다——
모두를 제물로 바치는 의식이 진정 신의 뜻이라면——
그럼, 나는 이 곳에서—— 신에게 선전포고를 하겠다.
[]
[]
2021-10-24 01:44:24
로드의 순수한 혈통이 아니라면 라그나로크를 사용하지 못하는 것은 당연하겠지/자동
[]
[]
2021-10-23 23:23:48
우린 라그나로크의 굴레에서 벗어났지. 아스가르드인들은 처음으로 자신들이 운명의 굴레에서 벗어나서 자유가 됐다고 믿고있고.
[]
[]
2021-10-23 22:16:51
.... ..... 어찌 됐든... 나는 이제 탈락자고, 당신은 살아남아있으니까... ... 살아남기를 바랄게. 벌써

### 데이터프레임 형태로 수집

In [10]:
# 크롤링된 데이터를 저장할 데이터 프레임입니다.
columns = ['created', 'tweet_text']
df = pd.DataFrame(columns=columns)

In [11]:
# 크롤링을 수행할 갯수를 입력하고, Cursor 객체를 사용하여 크롤링을 수행합니다.
max_tweets = 1000
searched_tweets = [status for status in tweepy.Cursor(api.search, q=keyword).items(max_tweets)]
#searched_tweets = [status for status in tweepy.Cursor(api.search_tweets, q=keyword).items(max_tweets)]  #For ver 4.0

In [12]:
# 키워드가 포함된 1000개의 트윗들에서, ‘text’, ‘created_at’ 정보를 데이터 프레임으로 저장합니다.
for tweet in searched_tweets:
    tweet_json = tweet._json
    tweet_text = tweet_json['text']
    created = tweet_json['created_at']
    row = [created, tweet_text]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)

In [13]:
# file name
# file_name = 'tweet_temp'
# rel_path =  output_path + file_name + '_' + str(now.tm_mon).zfill(2) + str(now.tm_mday).zfill(2) + str(now.tm_hour).zfill(2) + str(now.tm_min).zfill(2) + '.csv'

df.to_csv("tweet_temp.csv", index=False)

## <Step2. 추출> : 키워드 추출

### [텍스트 데이터 전처리]

In [14]:
df = pd.read_csv("tweet_temp.csv")
df.head()

,created,tweet_text
0,Sun Oct 24 04:36:09 +0000 2021,RT @MCU_0range_: 토르 라그나로크에서 로키랑 발키리 전투씬이 기억에 남...
1,Sun Oct 24 04:30:34 +0000 2021,토르 라그나로크에서 로키랑 발키리 전투씬이 기억에 남은 이유. 로키가 정신 지배 능...
2,Sun Oct 24 03:54:02 +0000 2021,왜.네가 이것을...라그나로크를 가지고 있는거지?/자동
3,Sun Oct 24 03:23:59 +0000 2021,라그나로크라고?어떻게..../자동
4,Sun Oct 24 02:40:01 +0000 2021,유독 라그나로크 비하인드가 많은 거 같은데... 가장 안 풀리긴 했지..... 저때...


In [15]:
# import library
import re # re — Regular expression operations

# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거합니다.
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글의 정규표현식을 나타냅니다.
    result = hangul.sub('', text)
    return result

In [16]:
# ‘tweet_text’ 피처에 이를 적용합니다.
df['ko_text'] = df['tweet_text'].apply(lambda x: text_cleaning(x))
df.head()

,created,tweet_text,ko_text
0,Sun Oct 24 04:36:09 +0000 2021,RT @MCU_0range_: 토르 라그나로크에서 로키랑 발키리 전투씬이 기억에 남...,토르 라그나로크에서 로키랑 발키리 전투씬이 기억에 남은 이유 로키가 정신 지배 ...
1,Sun Oct 24 04:30:34 +0000 2021,토르 라그나로크에서 로키랑 발키리 전투씬이 기억에 남은 이유. 로키가 정신 지배 능...,토르 라그나로크에서 로키랑 발키리 전투씬이 기억에 남은 이유 로키가 정신 지배 능력...
2,Sun Oct 24 03:54:02 +0000 2021,왜.네가 이것을...라그나로크를 가지고 있는거지?/자동,왜네가 이것을라그나로크를 가지고 있는거지자동
3,Sun Oct 24 03:23:59 +0000 2021,라그나로크라고?어떻게..../자동,라그나로크라고어떻게자동
4,Sun Oct 24 02:40:01 +0000 2021,유독 라그나로크 비하인드가 많은 거 같은데... 가장 안 풀리긴 했지..... 저때...,유독 라그나로크 비하인드가 많은 거 같은데 가장 안 풀리긴 했지 저때 망토 찢어져서...


### [konlpy를 이용한 키워드 추출]

In [17]:
# JPype1
#!pip install JPype1 #Successfully installed JPype1-1.3.0

In [18]:
# !pip install konply

# AttributeError: module 'tweepy' has no attribute 'StreamListener'
# 2021년 10월 6일 기준 진행 시 tweepy 버전 이슈 문제가 존재합니다.
# https://github.com/konlpy/konlpy/issues/371

In [19]:
tweepy.__version__

'3.10.0'

In [20]:
# import library
from konlpy.tag import Okt
from collections import Counter


# 한국어 약식 불용어사전 예시 파일입니다. 출처 - (https://www.ranks.nl/stopwords/korean)
korean_stopwords_path = "../data/korean_stopwords.txt"

with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]


In [21]:
def get_nouns(x):
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(x)
    
    # 한글자 키워드를 제거합니다.
    nouns = [noun for noun in nouns if len(noun) > 1]
    
    # 불용어를 제거합니다.
    nouns = [noun for noun in nouns if noun not in stopwords]
    
    return nouns

In [22]:
# ‘ko_text’ 피처에 이를 적용합니다.
df['nouns'] = df['ko_text'].apply(lambda x: get_nouns(x))
print(df.shape)
df.head()

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.